In [1]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [4]:
import pypdf


ModuleNotFoundError: No module named 'pypdf'

In [20]:
import chemdataextractor as cde

with open(".././mcs2021-lithium.pdf", 'rb') as f:
    doc = cde.Document.from_file(f, readers=[cde.reader.PdfReader()])

In [80]:
with open("/home/mevans/mcs2021.pdf", 'rb') as f:
    doc = cde.Document.from_file(f, readers=[cde.reader.PdfReader()])

In [195]:
text = "hEllo"
print(list(zip(text, [c.isalpha() ^ c.isupper() for c in text])))

[('h', True), ('E', False), ('l', True), ('l', True), ('o', True)]


In [330]:
import periodictable
from collections import defaultdict
names = [elem.name for elem in periodictable.elements]
elem_map = defaultdict(list)
last = None

def no_newline(text):
    return "\n" not in text

def any_alpha(text):
    return any(c.isalpha() for c in text)

def min_upper_alpha(text, min_=3):
    return sum([(c.isalpha() and c.isupper()) for c in text]) > min_

def all_alpha_are_upper(text):
    """Ensure that all alphabetical characters are upper-case.
    
    >>> all_alpha_are_upper("abcd")
    False
    >>> all_alpha_are_upper("ABCD")
    True
    >>> all_alpha_are_upper("AB1CD")
    True
    >>> all_alpha_are_upper("AB1Cd")
    False
    
    """
    return not any(c.isalpha() ^ c.isupper() for c in text)

def max_spaces(text, max_=5):
    return text.count(" ") < 5

def all_upper(text):
    return all(c.isupper() for c in text)

def is_commodity_header(text):
    return no_newline(text) and any_alpha(text) and min_upper_alpha(text) and all_alpha_are_upper(text) and max_spaces(text)


for ind, elem in enumerate(doc.elements):
    if "intentionally left blank" in elem.text.lower():
        start_ind = ind + 1
    if "appendix" in elem.text.lower() and is_commodity_header(elem.text):
        end_ind = ind -1
        break
        
import re

for ind, elem in enumerate(doc.elements[start_ind:end_ind]):
    if is_commodity_header(elem.text):
        elem_text = re.sub(r"[0-9]+", "", elem.text)
        if last != elem_text:
            elem_map[elem_text].append(start_ind + ind)
            if last is not None:
                elem_map[last].append(start_ind + ind - 1)
        last = elem_text
elem_map[last].append(end_ind)

In [331]:
non_elems = [elem for elem in elem_map if elem.lower() not in names]
for not_elem in non_elems:
    elem_map.pop(not_elem)

In [332]:
elem_map

defaultdict(list,
            {'ALUMINUM': [1039, 1114],
             'ANTIMONY': [1115, 1190],
             'ARSENIC': [1191, 1244],
             'BERYLLIUM': [1436, 1504],
             'BISMUTH': [1505, 1561],
             'BORON': [1562, 1630],
             'BROMINE': [1631, 1690],
             'CADMIUM': [1691, 1750],
             'CESIUM': [1805, 1834],
             'CHROMIUM': [1835, 1922],
             'COBALT': [2008, 2091],
             'COPPER': [2092, 2180],
             'GALLIUM': [2449, 2510],
             'GERMANIUM': [2622, 2691],
             'GOLD': [2692, 2748],
             'HELIUM': [2858, 2905],
             'INDIUM': [2906, 2955],
             'IODINE': [2956, 3012],
             'LEAD': [3399, 3466],
             'LITHIUM': [3507, 3588],
             'MANGANESE': [3696, 3784],
             'MERCURY': [3785, 3848],
             'MOLYBDENUM': [3936, 3990],
             'NICKEL': [3991, 4080],
             'RHENIUM': [4679, 4727],
             'RUBIDIUM': [4728, 475

In [340]:
def get_world_resource(element, elem_map, doc):
    d = doc[slice(*elem_map[element])]
    for ind, para in enumerate(d):
        print(para.text)
        if "Production and Reserves" in para.text or "Production and Capacity" in para.text:
            table_start = ind
        if para.text.startswith("World Resources:"):
            table_end = ind - 1
            resource = para
            break
    else:
        raise RuntimeError("Unable to find world resources paragraph")
        
    production_table = d[table_start:table_end]
    
    resource = nlp(para.text)
    reserve_guess = None
    for c in resource.ents:
        if c.label_ in ("QUANTITY" or "NUMBER"):
             print(f"Found quantity for {c} of {elem}")
             if reserve_guess is None or c > reserve_guess:
                reserve_guess = c
    if reserve_guess is None:
        print(f"Unable to find quantity for {elem}")
        
    return reserve_guess, production_table
        
resource = get_world_resource("ALUMINUM", elem_map, doc)

ALUMINUM1
(Data in thousand metric tons unless otherwise noted)
Domestic Production and Use: In 2020, three companies operated seven primary aluminum smelters in six States. 
Two smelters operated at full capacity and four smelters operated at reduced capacity throughout the year. One 
smelter operated at reduced capacity until it was idled in July. One other smelter remained on standby throughout the 
year. Domestic smelters were operating at about 49% of capacity of 1.79 million tons per year at yearend 2020. 
Production decreased by 8% after increasing in 2019. Based on published prices, the value of primary aluminum 
production was about $1.98 billion, 17% less than the value in 2019. The average annual U.S. market price declined 
by about 11% from that in 2019. Transportation applications accounted for 40% of domestic consumption; in 
descending order of consumption, the remainder was used in packaging, 21%; building, 14%; electrical, 8%; 
consumer durables, 7%; machinery, 7%; and

In [341]:
resource[-1]

[Paragraph(id=None, references=[], text='World Smelter Production and Capacity: Capacity data for China and Russia were revised based on Government \nand company data.'),
 Paragraph(id=None, references=[], text='United States \nAustralia \nBahrain \nCanada \nChina \nIceland \nIndia \nNorway \nRussia \nUnited Arab Emirates \nOther countries'),
 Paragraph(id=None, references=[], text='World total (rounded)'),
 Paragraph(id=None, references=[], text='Production'),
 Paragraph(id=None, references=[], text='2019 \n1,093 \n1,570 \n1,370 \n2,850 \n35,000 \n845 \n3,640 \n1,400 \n3,640 \n2,600 \n  9,200 \n63,200'),
 Paragraph(id=None, references=[], text='2020e \n1,000 \n1,600 \n1,500 \n3,100 \n37,000 \n840 \n3,600 \n1,400 \n3,600 \n2,600 \n  9,000 \n65,200'),
 Paragraph(id=None, references=[], text='Yearend capacity \n2020e \n1,790 \n1,720 \n1,540 \n3,270 \n43,000 \n890 \n4,060 \n1,430 \n4,020 \n2,700 \n12,300 \n76,700')]

In [334]:
elem_resource = {}
for elem in elem_map:
    elem_resource[elem] = get_world_resource(elem, elem_map, doc)

ALUMINUM
(between 55 billion and 75 billion tons,)
Found quantity for between 55 billion and 75 billion tons of ALUMINUM
ANTIMONY
(U.S., Alaska, Idaho, Montana, Nevada, Australia, Bolivia, China, Mexico, Russia, South Africa, Tajikistan, Mississippi Valley)
Unable to find quantity for ANTIMONY
ARSENIC
(Peru, Philippines, Chile, Canada, China, Guemassa, Marrakech, Morocco)
Unable to find quantity for ARSENIC
BERYLLIUM
(more than 100,000 tons, About 60%, the United States, the Spor Mountain area, Utah, the McCullough 
Butte area, Nevada, the Black Hills, South Dakota, the Sierra Blanca, Texas, the Seward Peninsula, the Gold Hill, Utah)
Found quantity for more than 100,000 tons of BERYLLIUM
BISMUTH
(China, Vietnam, the Tasna Mine, Bolivia, China, The Tasna Mine, 1996)
Unable to find quantity for BISMUTH
BORON
(the Mojave Desert, the United States, Alpide, Asia, South America, U.S., About 70%, Turkey)
Unable to find quantity for BORON
BROMINE
(The Dead Sea, the Middle East, 1 billion tons,

In [303]:
reserve_nlp.ents[0].label_

'QUANTITY'

In [283]:
reserve_nlp = nlp(reserves.text)

In [41]:
for elem in doc.elements:
    
    if elem.text.startswith("World Resources"):
        resources = elem if elem.text.startswith("World Resources") else None
        break

In [51]:
text = resources.text

In [55]:
parsed = nlp(text)

In [56]:
parsed

World Resources:5 Owing to continuing exploration, identified lithium resources have increased substantially 
worldwide and total about 86 million tons. Lithium resources in the United States—from continental brines, 
geothermal brines, hectorite, oilfield brines, and pegmatites—are 7.9 million tons. Lithium resources in other countries 
have been revised to 78 million tons. Lithium resources are Bolivia, 21 million tons; Argentina, 19.3 million tons; Chile, 
9.6 million tons; Australia, 6.4 million tons; China, 5.1 million tons; Congo (Kinshasa), 3 million tons; Canada, 
2.9 million tons; Germany, 2.7 million tons; Mexico, 1.7 million tons; Czechia, 1.3 million tons; Serbia, 1.2 million tons; 
Peru, 880,000 tons; Mali, 700,000 tons; Zimbabwe, 500,000 tons; Brazil, 470,000 tons; Spain, 300,000 tons; Portugal, 
270,000 tons; Ghana, 90,000 tons; and Austria, Finland, Kazakhstan, and Namibia, 50,000 tons each.

In [296]:
for ent in doc.ents:
    print(ent, ent.label)

AttributeError: 'Document' object has no attribute 'ents'

In [73]:
for c in doc.noun_chunks:
    print(c.text, c.root.text, c.root.dep_, c.root.head.text)

The identified lithium resources resources nsubj total
760,000 tons tons dobj total
the United States States pobj in
more than 12 million tons tons conj total
other countries countries pobj in


In [72]:
for token in doc:
    print(token.text, token.dep_, token.head.text, [c for c in token.children])
    if token.

The det resources []
identified amod resources []
lithium compound resources []
resources nsubj total [The, identified, lithium]
total ROOT total [resources, tons, in, and, tons, .]
760,000 nummod tons []
tons dobj total [760,000]
in prep total [States]
the det States []
United compound States []
States pobj in [the, United]
and cc total []
more amod million []
than quantmod million []
12 compound million []
million nummod tons [more, than, 12]
tons conj total [million, in]
in prep tons [countries]
other amod countries []
countries pobj in [other]
. punct total []


In [70]:
doc = nlp("The identified lithium resources total 760,000 tons in the United States and more than 12 million tons in other countries.")

In [71]:
doc.

SyntaxError: invalid syntax (<ipython-input-71-934ff3e6d73d>, line 1)